In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn import svm
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

/home/sakuragi/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
#Read CSV, index+1 supaya index dimulai dari 1, tidak 0, memudahkan pembacaan dataframe dibawah.
df = pd.read_csv('diabetes.csv')
df.index += 1

In [3]:
#melihat dataframe diabetes.csv
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,diabetes
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1


In [4]:
#target / label. Kolom terakhir/kolom 'diabetes'.
#mengambil dataframe untuk disimpan menjadi array. 
#np_target = df.iloc[:,-1:].values #alternatif
np_target = df['diabetes'].values

In [5]:
#Features. Mengambil kolom 1 - 8 dengan mengakses index. 
np_data = df.iloc[:,:-1].values

In [6]:
#Support Vector Classification model
clf = svm.SVC(kernel='linear', gamma=0.001, C=100)

In [7]:
#Split dataset, 20% testing 80% training.
X_train, X_test, y_train, y_test = train_test_split(np_data,np_target,test_size=0.2, random_state=None)

In [8]:
X_train

array([[1.000e+01, 1.480e+02, 8.400e+01, ..., 3.760e+01, 1.001e+00,
        5.100e+01],
       [1.300e+01, 1.520e+02, 9.000e+01, ..., 2.680e+01, 7.310e-01,
        4.300e+01],
       [1.000e+00, 1.090e+02, 5.800e+01, ..., 2.850e+01, 2.190e-01,
        2.200e+01],
       ...,
       [3.000e+00, 1.580e+02, 7.600e+01, ..., 3.160e+01, 8.510e-01,
        2.800e+01],
       [3.000e+00, 9.900e+01, 5.400e+01, ..., 2.560e+01, 1.540e-01,
        2.400e+01],
       [8.000e+00, 1.050e+02, 1.000e+02, ..., 4.330e+01, 2.390e-01,
        4.500e+01]])

In [9]:
#Cuma buat liat ukuran data train vs data test
len(X_train), len(X_test)

(614, 154)

In [10]:
#Fit model SVM sesuai data training (proses training model)
clf.fit(X_train, y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
#CrossValidation model yang sudah dibangun. Dengan parameter scoring=accuracy,precision, dan recall
scoring = ['accuracy', 'precision', 'recall']
scores = cross_validate(clf, X_test, y_test, scoring=scoring, cv=5, return_train_score=True)

In [12]:
#Menampilkan hasil dari cross validation
scores

{'fit_time': array([18.47057104,  8.19345355, 24.70905519, 50.10510302, 19.49726129]),
 'score_time': array([0.00218058, 0.00207448, 0.00210571, 0.00207114, 0.00208282]),
 'test_accuracy': array([0.58064516, 0.87096774, 0.70967742, 0.80645161, 0.73333333]),
 'train_accuracy': array([0.80487805, 0.77235772, 0.80487805, 0.78861789, 0.79032258]),
 'test_precision': array([0.375     , 0.76923077, 0.66666667, 0.69230769, 0.63636364]),
 'train_precision': array([0.73809524, 0.71052632, 0.73809524, 0.73684211, 0.70454545]),
 'test_recall': array([0.27272727, 0.90909091, 0.36363636, 0.81818182, 0.63636364]),
 'train_recall': array([0.70454545, 0.61363636, 0.70454545, 0.63636364, 0.70454545])}

In [13]:
#menampilkan test_accuracy dari cross validation, memiliki 5 value, tiap value adalah representasi akurasi dari tiap iterasi
scores['test_accuracy']

array([0.58064516, 0.87096774, 0.70967742, 0.80645161, 0.73333333])

In [14]:
#Hasil cross validation berupa dictionary. Kemudian dictionary disimpan menjadi dataframe. Hanya untuk keperluan pembacaan
df_scores = pd.DataFrame.from_dict(scores)
df_scores.index +=1
df_scores

,fit_time,score_time,test_accuracy,train_accuracy,test_precision,train_precision,test_recall,train_recall
1,18.470571,0.002181,0.580645,0.804878,0.375000,0.738095,0.272727,0.704545
2,8.193454,0.002074,0.870968,0.772358,0.769231,0.710526,0.909091,0.613636
3,24.709055,0.002106,0.709677,0.804878,0.666667,0.738095,0.363636,0.704545
4,50.105103,0.002071,0.806452,0.788618,0.692308,0.736842,0.818182,0.636364
5,19.497261,0.002083,0.733333,0.790323,0.636364,0.704545,0.636364,0.704545


In [16]:
#DUMP MODEL YANG SUDAH DIBUAT AGAR MENJADI MODEL PERSISTENCE DAN LANGSUNG DIGUNAKAN DITEMPAT LAIN.
joblib.dump(clf, 'praktikum1_svc.pkl') 

['praktikum1_svc.pkl']

In [16]:
pickle.dump(clf, open('model.pkl','wb'))